In [1]:
import papermill as pm
import duckdb, subprocess, os, time
from joblib import Parallel, delayed

In [2]:
con = duckdb.connect()
con.install_extension("spatial")
con.load_extension("spatial")
con.install_extension("azure")
con.load_extension("azure")

In [ ]:
start_time = time.time()
print(time.ctime(time.time()))

In [3]:
pm.inspect_notebook('WaterfowlHabitatSinglefips.ipynb')

{'inaoifile': {'name': 'inaoifile',
  'inferred_type_name': 'None',
  'default': "'28027'",
  'help': ''},
 'local': {'name': 'local',
  'inferred_type_name': 'None',
  'default': 'True',
  'help': ''}}

In [4]:
con.sql("SET azure_storage_connection_string = 'DefaultEndpointsProtocol=https;AccountName=giscog;EndpointSuffix=core.windows.net';")
con.sql(f"""
CREATE OR REPLACE TABLE counties AS
SELECT NAME, STATE_NAME, FIPS, geometry FROM read_parquet('azure://abdu/uscounties.parquet')
WHERE STATE_NAME = 'Mississippi'
""")

In [5]:
allfips = con.sql('select distinct(fips) from counties').df().values.tolist()

In [6]:
listfips = sorted([item for items in allfips for item in items])
print(listfips)

['28001', '28003', '28005', '28007', '28009', '28011', '28013', '28015', '28017', '28019', '28021', '28023', '28025', '28027', '28029', '28031', '28033', '28035', '28037', '28039', '28041', '28043', '28045', '28047', '28049', '28051', '28053', '28055', '28057', '28059', '28061', '28063', '28065', '28067', '28069', '28071', '28073', '28075', '28077', '28079', '28081', '28083', '28085', '28087', '28089', '28091', '28093', '28095', '28097', '28099', '28101', '28103', '28105', '28107', '28109', '28111', '28113', '28115', '28117', '28119', '28121', '28123', '28125', '28127', '28129', '28131', '28133', '28135', '28137', '28139', '28141', '28143', '28145', '28147', '28149', '28151', '28153', '28155', '28157', '28159', '28161', '28163']


In [17]:
def f(x):
    try:
        if os.path.isfile('./output/'+x+'.parquet'):
            return (x, 'exists')
        else:
            subprocess.run(pm.execute_notebook('WaterfowlHabitatSinglefips.ipynb','./output/{0}.ipynb'.format(x),parameters=dict(inaoifile=x, local=True)), shell=True)
            return (x, 'run complete')
    except Exception as e:
        if os.path.isfile('./output/'+x+'.parquet'):
            return (x, 'run complete')
        return (x,'fail', e)

In [18]:
completed = Parallel(n_jobs=8, verbose=10)(delayed(f)(listfips[x]) for x in range(len(listfips)))

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed: 21.3min
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed: 50.5min
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed: 89.5min
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed: 167.7min
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed: 231.2min
[Parallel(n_jobs=8)]: Done  45 tasks      | elapsed: 296.2min
[Parallel(n_jobs=8)]: Done  56 tasks      | elapsed: 367.8min
[Parallel(n_jobs=8)]: Done  76 out of  82 | elapsed: 462.0min remaining: 36.5min
[Parallel(n_jobs=8)]: Done  82 out of  82 | elapsed: 511.8min finished


In [19]:
#check for failed
completed
#fixme = completed

[('28001', 'run complete'),
 ('28003', 'run complete'),
 ('28005', 'run complete'),
 ('28007', 'run complete'),
 ('28009', 'run complete'),
 ('28011', 'run complete'),
 ('28013', 'run complete'),
 ('28015', 'run complete'),
 ('28017', 'run complete'),
 ('28019', 'run complete'),
 ('28021', 'run complete'),
 ('28023', 'run complete'),
 ('28025', 'run complete'),
 ('28027', 'run complete'),
 ('28029', 'run complete'),
 ('28031', 'run complete'),
 ('28033', 'run complete'),
 ('28035', 'run complete'),
 ('28037', 'run complete'),
 ('28039', 'run complete'),
 ('28041', 'run complete'),
 ('28043', 'run complete'),
 ('28045', 'run complete'),
 ('28047', 'run complete'),
 ('28049', 'run complete'),
 ('28051', 'run complete'),
 ('28053', 'run complete'),
 ('28055', 'run complete'),
 ('28057', 'run complete'),
 ('28059', 'run complete'),
 ('28061', 'run complete'),
 ('28063', 'run complete'),
 ('28065', 'run complete'),
 ('28067', 'run complete'),
 ('28069', 'run complete'),
 ('28071', 'run comp

In [9]:
fixme = ['28135']
completed = Parallel(n_jobs=4, verbose=10)(delayed(f)(fixme[x]) for x in range(len(fixme)))

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed: 20.6min


In [10]:
completed

[('28135', 'run complete')]

In [ ]:
print('Done in {0:.1f} seconds'.format(time.time() - start_time))

In [ ]:
#merge all parquet files in ./output/*.parquet